## Introduction
#### The ***2024* Formula 1 season** marked a period of **competitive tightening within the mature ground-effect regulations**, with **Max Verstappen and Red Bull Racing** remaining the benchmark but facing **increasing pressure from Ferrari, McLaren, and Mercedes**, as aerodynamic convergence, tyre management improvements, and operational execution narrowed performance gaps; the season was characterised by **greater race-to-race variability**, strategic complexity, and midfield teams occasionally challenging the front under specific conditions, making 2024 a year defined by **reduced dominance**, **strategic nuance**, and a grid operating closer to its regulatory ceiling ahead of the major 2026 reset.

## Importing the necessary libraries

In [1]:
import pandas as pd
import fastf1
import logging

In [2]:
fastf1.Cache.enable_cache("../cache")

## Getting the Races of 2024


In [3]:
schedule = fastf1.get_event_schedule(2024)
listOfRaces = schedule["Country"].tolist()
listOfRaces

['Bahrain',
 'Bahrain',
 'Saudi Arabia',
 'Australia',
 'Japan',
 'China',
 'United States',
 'Italy',
 'Monaco',
 'Canada',
 'Spain',
 'Austria',
 'United Kingdom',
 'Hungary',
 'Belgium',
 'Netherlands',
 'Italy',
 'Azerbaijan',
 'Singapore',
 'United States',
 'Mexico',
 'Brazil',
 'United States',
 'Qatar',
 'United Arab Emirates']

## Iterating into every Race and concatenating every Race Data into a DataFrame
Through my first try in ["Bahrain_test.ipynb"](https://github.com/Chracker24/MTS-IE/blob/main/02_Notebooks/Data_Collection/Formula1/2020/Bahrain_test.ipynb), I am going to use a loop to collect data and trim it down to the column and data points that I deem necessary for the Intelligence Engine.

In [4]:
cols = [
    "Driver",
    "LapTime",
    "LapNumber",
    "Stint",
    "Sector1Time",
    "Sector2Time",
    "Sector3Time",
    "Compound",
    "Team",
    "Deleted",
]

In [5]:
Season_Data2024 = []

In [6]:
logging.getLogger("fastf1").setLevel(logging.ERROR)
for i in range(len(listOfRaces)):
    session = fastf1.get_session(2024, listOfRaces[i], "R")
    session.load(laps=True, telemetry=False, weather=False)
    laps = session.laps
    Race_Data = laps[cols].copy()
    Race_Data = Race_Data.dropna(subset=["LapTime"]).copy()
    Race_Data["Season"] = 2024
    Race_Data["Race"] = listOfRaces[i]
    Race_Data[["LapTime","Sector1Time","Sector2Time","Sector3Time"]] = Race_Data[["LapTime","Sector1Time","Sector2Time","Sector3Time"]].apply(lambda x : x.dt.total_seconds())
    Race_Data = Race_Data[Race_Data["Deleted"]==False].copy()

    Season_Data2024.append(Race_Data)

In [7]:
season_2024 = pd.concat(Season_Data2024, ignore_index=True)

In [8]:
season_2024 = season_2024.sort_values("LapNumber").reset_index(drop=True)
season_2024

,Driver,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,Compound,Team,Deleted,Season,Race
0,LAW,108.974,1.0,1.0,NaN,41.757,34.744,HARD,RB,False,2024,United States
1,LEC,99.663,1.0,1.0,NaN,38.818,25.950,MEDIUM,Ferrari,False,2024,United Kingdom
2,PIA,114.248,1.0,1.0,NaN,50.480,29.643,MEDIUM,McLaren,False,2024,Belgium
3,HUL,107.041,1.0,1.0,NaN,31.163,45.012,MEDIUM,Haas F1 Team,False,2024,China
4,ALO,92.411,1.0,1.0,NaN,32.154,29.198,MEDIUM,Aston Martin,False,2024,Italy
...,...,...,...,...,...,...,...,...,...,...,...,...
26895,RUS,76.303,78.0,2.0,19.846,35.851,20.606,MEDIUM,Mercedes,False,2024,Monaco
26896,PIA,77.126,78.0,2.0,19.954,36.457,20.715,HARD,McLaren,False,2024,Monaco
26897,SAI,77.072,78.0,2.0,20.086,36.430,20.556,HARD,Ferrari,False,2024,Monaco
26898,LEC,78.636,78.0,2.0,20.278,36.319,22.039,HARD,Ferrari,False,2024,Monaco


## Adding the Dataframe as a CSV file in Data/Archive

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[3]
archive_dir = PROJECT_ROOT / "Data" / "Archive"

archive_dir.mkdir(parents=True, exist_ok=True)

season_2024.to_csv(
    archive_dir / "season_2024.csv",
    index=False
)